# New data set
- ~10.400 domains
- time: ~14 hrs
- size: 14 GB (1.5 MB / embedding)
- Speed up through:
    - ordering protein sequences by length
    - sequences with roughly the same length are computed in the same batches (size 64)
    - this way the padding that had to be added is minimized

In [2]:
import os.path

import numpy as np

path = "../data/embeddings/protein_embeddings"
file = "1iq4A00.npy"

emb = np.load(os.path.join(path, file))


In [4]:
emb.shape

(179, 1024)

In [6]:
import pandas as pd

dataset = pd.read_csv("../data/subset_protein_mapped.csv")
dataset['protein_sequence_length'] = dataset['protein_sequence'].str.len()


In [7]:
dataset.sort_values("protein_sequence_length", ascending=False, inplace=True)

In [8]:
dataset


,domain_id,class,architecture,topology,homology,length,cath_domain_start,cath_domain_end,protein_sequence,domain_start,domain_end,domain_sequence,protein_sequence_length
2417,4rh7A01,1,20,920,20,415.0,2435.0,2849.0,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,1577.0,1991.0,REQLQTIYGAYLEPVLHKNLKNHSIWGSSSKIYLLAGSMVQVYEQV...,3450
2418,3vkhA11,1,20,920,20,342.0,3257.0,3598.0,MTRHHHHHHGGGDYKDDDDKGGGKVPVEEEIQDLKAVWVELSNTWQ...,1894.0,2235.0,PRHYLDFINQVVLLINEKRDQLEEEQLHLNIGLKKLRDTEAQVKDL...,3367
1353,7tzoA01,1,10,1070,11,174.0,2014.0,2187.0,MVTTLSGLSGEQGPSGDMTTEEDSATHIKFSKRDEDGRELAGATME...,2139.0,2312.0,EELIRVAILWHEMWHEGLEEASRLYFGERNVKGMFEVLEPLHAMME...,2674
5312,2vz8A04,3,10,129,110,254.0,857.0,1110.0,MEEVVIAGMSGKLPESENLEEFWANLIGGVDMVTADDRRWKAGLYG...,857.0,1110.0,SSVAVYKFDVSPESPDHYLVDHCIDGRVLFPGTGYLWLTWKTLARA...,2512
3108,7v1nA02,2,10,270,10,113.0,1977.0,2089.0,MSLVNRKQLEKMANVRFRVQEDEYVAILDALEEYHNMSENTVVEKY...,1977.0,2089.0,KFYFNSDGIMQKGFVNINDKTFYFDDSGVMKSGYTEIDGKYFYFAE...,2375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5892,3e7rL01,3,30,30,10,30.0,11.0,40.0,GFGCNGPWDEDDMQCHNHCKSIKGYKGGYCAKGGFVCKCY,11.0,40.0,DDMQCHNHCKSIKGYKGGYCAKGGFVCKCY,40
3167,2ysgA01,2,20,70,10,33.0,8.0,40.0,GSSGSSGLPYGWEEAYTADGIKYFINHVTQTTSWIHPVMS,8.0,40.0,LPYGWEEAYTADGIKYFINHVTQTTSWIHPVMS,40
2355,2l9uA00,1,20,890,10,40.0,637.0,676.0,MGRTHLTMALTVIAGLVVIFMMLGGTFLYWRGRRHHHHHH,1.0,40.0,MGRTHLTMALTVIAGLVVIFMMLGGTFLYWRGRRHHHHHH,40
1891,4il6X00,1,20,5,510,40.0,2.0,41.0,TITPSLKGFFIGLLSGAVVLGLTFAVLIAISQIDKVQRSL,1.0,40.0,TITPSLKGFFIGLLSGAVVLGLTFAVLIAISQIDKVQRSL,40
